In [1]:
import cv2
from IPython.display import clear_output, Image, display, HTML
import numpy as np
import time

In [3]:
def get_factors(n):
    list1=list()
    for i in range(1,n+1):
        if n%i==0:
            list1.append(i)
    return(list1)
def get_duration(direction, frame_width, frame_height, duration):
    if direction=="lr":
        factors=get_factors(frame_width)
        return factors[np.argmin(np.abs(np.array(factors)-duration*24))]
    if direction=="ud":
        factors=get_factors(frame_height)
        return factors[np.argmin(np.abs(np.array(factors)-duration*24))]
def get_settings(frame_width, frame_height):
        #get user settings
    filename = input("What would you like name this file?\n")
    file = cv2.VideoWriter('{}.avi'.format(filename),cv2.VideoWriter_fourcc('M','J','P','G'), 24, (frame_width, frame_height))
    direction = input("Which way should the blue line move? Enter \'lr\' for left right or \'ud\' for up down.\n")
    while direction!='lr' and direction!='ud':
        direction = input("I'm sorry that is an invalid response. Enter \'lr\' for left or \'ud\' for up down.\n")
    duration = int(input("For how many seconds would you like to record?\n"))
    duration = get_duration(direction, frame_width, frame_height, duration)
    record = input("Because we are limited by the dimensions of the webcam, the video will actually be {} seconds long.\n"
                   .format(np.round(duration/24,2))+
                   "Would you like to continue? Enter \'y\' for yes and \'n\' for n.\n")
    while record!='n' and record!='y':
        record = input("I'm sorry that is an invalid response. Enter \'y\' for yes or and \'n\' for no.\n")
    while record=='n':
        duration = int(input("For how many seconds would you like to record?\n"))
        duration = get_duration(direction, frame_width, frame_height, duration)
        record = input("Because we are limited by the dimensions of the webcam, the video will actually be {} seconds long. \n"
                       .format(np.round(duration/24,2))+
                   "Would you like to continue? Enter \'y\' for yes and \'n\' for n.\n")
        while record!='n' and record!='y':
            direction = input("I'm sorry that is an invalid response. Enter \'y\' for yes or and \'n\' for no.\n")
    clear_output()
    return (file, direction, duration)

In [ ]:
def get_vid():
    # initialize camera
    cam = cv2.VideoCapture(0)
    frame_width = int(cam.get(3))
    frame_height = int(cam.get(4))
    # grabs user input to configure time warp scan
    file, direction, duration = get_settings(frame_width, frame_height)
    print("You have two seconds to get in position.")
    time.sleep(2)
    clear_output()
    if direction=="lr":
        snip_width=int(frame_width/duration)
        frame_count=1
        display_handle=display(None, display_id=True)
        while frame_count <= duration:
            frame = cam.read()[1][:,::-1,:]
            if not frame_count-1:
                store=frame[:,0:frame_count*snip_width,:]
            else:
                store=np.append(store, frame[:,(frame_count-1)*snip_width:frame_count*snip_width,:],axis=1)
            frame[:,:frame_count*snip_width,:]=store
            frame[:,frame_count*snip_width:(frame_count+1)*snip_width,:]=[209, 199, 50]
            file.write(frame)
            display_handle.update(Image(data=cv2.imencode('.jpeg', frame)[1]))
            frame_count+=1
    if direction=="ud":
        snip_width=int(frame_height/duration)
        frame_count=1
        display_handle=display(None, display_id=True)
        while frame_count <= duration:
            frame = cam.read()[1][:,::-1,:]
            if not frame_count-1:
                store=frame[0:frame_count*snip_width,:,:]
            else:
                store=np.append(store, frame[(frame_count-1)*snip_width:frame_count*snip_width,:,:],axis=0)
            frame[:frame_count*snip_width,:,:]=store
            frame[frame_count*snip_width:(frame_count+1)*snip_width,:,:]=[209, 199, 50]
            file.write(frame)
            display_handle.update(Image(data=cv2.imencode('.jpeg', frame)[1]))
            frame_count+=1
    cam.release()
    file.release()
    display_handle.update(Image(data=cv2.imencode('.jpeg', store)[1]))
    return store

In [ ]:
store=get_vid()